In [70]:
import numpy as np
import pandas as pd
import requests
import zipfile
import io
%matplotlib inline
# Set File Paths https://usfca.qualtrics.com/admin/account-settings-portal/user-settings
path = "C:\\Users\\apagano\\OneDrive - JBS International\\Desktop\\"
survey_id = "SV_eUHQR9fgYjK9OWW"
api_token = "PjXVwK08ZwhJbDwKqMSAPvXqNXwKNW6HxcU67al1"  # API Token (DO NOT SHARE PUBLICLY)
csv_file_path = path + "Survey for Ukr.csv"

# Function to Download Data from Qualtrics
def get_qualtrics_survey(dir_save_survey, survey_id, api_token):
    """Automatically query the Qualtrics survey data and save it as a CSV."""
    file_format = "csv"
    data_center = 'usfca'  # Adjust based on your Qualtrics domain

    base_url = f"https://{data_center}.qualtrics.com/API/v3/responseexports/"
    headers = {"content-type": "application/json", "x-api-token": api_token}

    # Step 1: Creating Data Export
    #payload = f'{{"format":"{file_format}","surveyId":"{survey_id}"}}'
    payload = f'''{{
    "format":"csv",
    "surveyId":"{survey_id}",
    "useLabels": true

}}'''

    response = requests.post(base_url, data=payload, headers=headers)

    if response.status_code != 200:
        print("Error: Failed to create data export")
        print(response.text)
        return

    progress_id = response.json()["result"]["id"]
    print(f"Data export request sent. ID: {progress_id}")

    # Step 2: Checking Export Progress
    while True:
        check_url = base_url + progress_id
        check_response = requests.get(check_url, headers=headers)
        progress = check_response.json()["result"]["percentComplete"]
        if progress == 100:
            break

    print("Export complete. Downloading file...")

    # Step 3: Downloading and Extracting Data
    download_url = base_url + progress_id + '/file'
    download_response = requests.get(download_url, headers=headers, stream=True)

    zipfile.ZipFile(io.BytesIO(download_response.content)).extractall(dir_save_survey)
    print('✅ Downloaded Qualtrics survey successfully')

# Download Survey Data
get_qualtrics_survey(dir_save_survey=path, survey_id=survey_id, api_token=api_token)

# Read CSV into DataFrame
#df = pd.read_csv(csv_file_path)
df = pd.read_csv(csv_file_path, encoding="utf-8-sig")
print("CSV Loaded. First few rows:\n", df.head())
print("Column Data Types:\n", df.dtypes)



Data export request sent. ID: ES_u40ne3eko8vg9aio6k7kpcsvvv
Export complete. Downloading file...
✅ Downloaded Qualtrics survey successfully


C:\Users\apagano\AppData\Local\Temp\ipykernel_2044\786344710.py:63: DtypeWarning: Columns (1,5,6,7,8,9,10,11,12,13,14,16,17,18,19,20,21,22,23,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59,60,61,62,63,64,69,70,71,72,73,76,77,78,79,80,81,82,83,84,85,86,91,92,98,101,104,106,108,109,110,115,116,118,119,120,121,122,123,124,127,128,129,132,133,134,135,136,137,138,139,140,141,142,143,145,149,150,157,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,289,297,303,319,320,321,322,324,325,326,327,329,331,333,335,

CSV Loaded. First few rows:
                    ResponseID                    ResponseSet  \
0                  ResponseID                    ResponseSet   
1  {'ImportId': 'responseId'}  {'ImportId': 'responseSetId'}   
2           R_1BVgP7O3xvP21VY                            NaN   
3           R_21Hz041GY9SpKri                            NaN   
4           R_2pS2vzuQN9r4wD9                            NaN   

                   IPAddress                  StartDate  \
0                  IPAddress                  StartDate   
1  {'ImportId': 'ipAddress'}  {'ImportId': 'startDate'}   
2             178.154.148.86        2023-08-07 00:55:15   
3               141.8.129.86        2023-08-07 03:24:02   
4               92.38.148.58        2023-08-14 05:56:39   

                   EndDate                        RecipientLastName  \
0                  EndDate                        RecipientLastName   
1  {'ImportId': 'endDate'}  {'ImportId': 'panel-RecipientLastName'}   
2      2023-08-07 

In [71]:

df = pd.DataFrame(df)

def unify_numeric_strings(col):
    def convert(val):
        try:
            return str(float(val))  # forces decimal format
        except:
            return str(val)  # keep non-numeric as string
    return col.map(convert)




In [72]:
print(df['Finished'].unique())

['Finished' "{'ImportId': 'finished'}" '0' '1' 1 0]


In [73]:
df2 = df.iloc[2:].reset_index(drop=True)
df2 = df2.apply(unify_numeric_strings)
print(df2['Finished'].unique())
#df2 = df2[df2["Finished"] == "1"]
df2 = df2[df2["Age"].notna()]
df2 = df2[df2["Q891"].notna()]
df2 = df2[df2["assignment_id"].notna()]
df2 = df2[df2["completion_code"].notna()]
df2 = df2[df2["ResponseID"].notna()]
df2 = df2[df2["response_id"].notna()]
df2 = df2[~df2["assignment_id"].isin(["undefined"])]
df2 = df2[~df2["completion_code"].isin(["undefined"])]
df2 = df2[~df2["ResponseID"].isin(["undefined"])]
df2 = df2[~df2["response_id"].isin(["undefined"])]

print(df2['Finished'].unique())
print(len(df2))


['0.0' '1.0']
['1.0' '0.0']
1184


In [74]:

df2["numericDate"] = pd.to_datetime(
    df2["EndDate"],
    format="%Y-%m-%d %H:%M:%S"
)

df2 = df2.sort_values("numericDate")
df2 = df2.drop_duplicates(subset=["completion_code"])
df2 = df2.drop_duplicates(subset=["assignment_id"])
df2 = df2.drop_duplicates(subset=["ResponseID"])
df2 = df2.drop_duplicates(subset=["response_id"])

print(df2['Finished'].unique())
print(len(df2))


['0.0' '1.0']
916


In [75]:
# List all column names
print(df2.columns.tolist())




['ResponseID', 'ResponseSet', 'IPAddress', 'StartDate', 'EndDate', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalDataReference', 'Finished', 'Status', 'Score-sum', 'Score-weightedAvg', 'Score-weightedStdDev', 'response_id', 'assignment_id', 'besample_id', 'completion_code', 'Q_BallotBoxStuffing', 'Q_RecaptchaScore', 'Q_RelevantIDDuplicate', 'Q_RelevantIDDuplicateScore', 'Q_RelevantIDFraudScore', 'manuallyIDduplicateIP', 'Q_selection', 'selection', 'Survey, Treatment, Prosocial', 'Q_TotalDuration', 'StartLoop3', 'elapsed3', 'timeout3', 'Score3', 'StartLoop4', 'elapsed4', 'timeout4', 'Score4', 'StartLoop5', 'elapsed5', 'timeout5', 'Score5', 'StartLoop', 'elapsed', 'timeout', 'Score', 'StartLoop2', 'elapsed2', 'timeout2', 'Score2', 'StartLoop6', 'elapsed6', 'timeout6', 'Score6', 'Surveys, Control, Prosocial', 'StartLoop7', 'elapsed7', 'timeout7', 'Score7', 'StartLoop8', 'elapsed8', 'timeout8', 'Score8', 'StartLoop9', 'elapsed9', 'timeout9', 'Score9', 'Treatment, Pr

In [76]:
df2 = df2.drop_duplicates(subset=[
    "IPAddress",
    "Q3051_1_TEXT",
    "Q3051_2_TEXT",
    "Q3051_3_TEXT",
    "Q3051_4_TEXT",
    "LocationLatitude",
    "LocationLongitude"
])
print(len(df2))

771


In [77]:
for var in df2.columns:
    print(var)

ResponseID
ResponseSet
IPAddress
StartDate
EndDate
RecipientLastName
RecipientFirstName
RecipientEmail
ExternalDataReference
Finished
Status
Score-sum
Score-weightedAvg
Score-weightedStdDev
response_id
assignment_id
besample_id
completion_code
Q_BallotBoxStuffing
Q_RecaptchaScore
Q_RelevantIDDuplicate
Q_RelevantIDDuplicateScore
Q_RelevantIDFraudScore
manuallyIDduplicateIP
Q_selection
selection
Survey, Treatment, Prosocial
Q_TotalDuration
StartLoop3
elapsed3
timeout3
Score3
StartLoop4
elapsed4
timeout4
Score4
StartLoop5
elapsed5
timeout5
Score5
StartLoop
elapsed
timeout
Score
StartLoop2
elapsed2
timeout2
Score2
StartLoop6
elapsed6
timeout6
Score6
Surveys, Control, Prosocial
StartLoop7
elapsed7
timeout7
Score7
StartLoop8
elapsed8
timeout8
Score8
StartLoop9
elapsed9
timeout9
Score9
Treatment, Prosocial, Surveys
Control, Prosocial, Surveys
Prosocial, Treatment, Surveys
Prosocial, Control, Surveys
completed
ChildAnxiety
ChildDepression
TScoreA
TScoreD
ASeverity
DSeverity
Q3026
Intro
Q3051_1

In [78]:
# Drop empty columns
print(len(df2.columns))
df2 = df2.dropna(axis=1, how='all')
print(len(df2.columns))
print(df2['ResponseSet'].unique())
cols_to_drop = [col for col in df2.columns 
                if df2[col].nunique(dropna=False) == 1 and df2[col].iloc[0] == 'nan']
df2 = df2.drop(columns=cols_to_drop)
print(len(df2.columns))


8623
8623
['nan']
4010


In [79]:
print(len(df2.columns))

4010


In [80]:

# Drop nonuseful columns where there is only one unique value


cols_to_drop = []

for col in df2.columns:
    unique_vals = df2[col].unique()
    if len(unique_vals) == 1:
        print(f"Column: {col}, Unique value: {unique_vals[0]}")
        cols_to_drop.append(col)

# Step 2: print them
print("Dropping columns:", cols_to_drop)

# Step 3: drop them
df2 = df2.drop(columns=cols_to_drop)
print(len(df2.columns))

Column: Status, Unique value: 0.0
Column: Score-sum, Unique value: 0.0
Column: Score-weightedAvg, Unique value: 0.0
Column: Score-weightedStdDev, Unique value: 0.0
Column: besample_id, Unique value: 1033.0
Column: Q3026, Unique value: 1.0
Column: LocationAccuracy, Unique value: -1.0
Dropping columns: ['Status', 'Score-sum', 'Score-weightedAvg', 'Score-weightedStdDev', 'besample_id', 'Q3026', 'LocationAccuracy']
4003


In [81]:
# Drop specific identified useless columns
df2 = df2.drop(columns=[col for col in df2.columns if col.startswith("DO-Q-")])


In [82]:
print(df2.columns.tolist())


cols_2_val = []
for col in df2.columns:
    unique_vals = df2[col].unique()
    print(f"Column: {col}, Unique values: {len(unique_vals)}, First 3: {unique_vals[:3]}")
    if len(unique_vals) == 2:
        cols_2_val.append(col)

['ResponseID', 'IPAddress', 'StartDate', 'EndDate', 'Finished', 'response_id', 'assignment_id', 'completion_code', 'Q_RecaptchaScore', 'Q_RelevantIDDuplicate', 'Q_RelevantIDDuplicateScore', 'Q_RelevantIDFraudScore', 'manuallyIDduplicateIP', 'selection', 'Survey, Treatment, Prosocial', 'Q_TotalDuration', 'StartLoop3', 'elapsed3', 'timeout3', 'Score3', 'StartLoop4', 'elapsed4', 'timeout4', 'Score4', 'StartLoop5', 'elapsed5', 'timeout5', 'Score5', 'StartLoop', 'elapsed', 'timeout', 'Score', 'StartLoop2', 'elapsed2', 'timeout2', 'Score2', 'StartLoop6', 'elapsed6', 'timeout6', 'Score6', 'Surveys, Control, Prosocial', 'StartLoop7', 'elapsed7', 'timeout7', 'Score7', 'StartLoop8', 'elapsed8', 'timeout8', 'Score8', 'StartLoop9', 'elapsed9', 'timeout9', 'Score9', 'Treatment, Prosocial, Surveys', 'Control, Prosocial, Surveys', 'Prosocial, Treatment, Surveys', 'Prosocial, Control, Surveys', 'completed', 'ChildAnxiety', 'ChildDepression', 'TScoreA', 'TScoreD', 'ASeverity', 'DSeverity', 'Intro', 'Q3

In [83]:
df3=df2

In [84]:
child_cols = [col for col in df3.columns if (col.startswith("Child") or col.startswith("PRCADS"))]
child_cols_todrop = child_cols
df_S_STR_DG = df3.drop(columns=child_cols_todrop)


child_cols.append('ResponseID')
df_child = df3[child_cols]

print(child_cols) 
print(f"Child: len({len(df_child.columns)})")
print(f"Adult: len({len(df_S_STR_DG.columns)})")

for item in df_S_STR_DG.columns:
    print(item)

['ChildAnxiety', 'ChildDepression', 'Child War Start(1)', 'Child War Start_TEXT(1)', 'Child Age\xa0(1)', 'Child Gender\xa0(1)', 'Child Grade\xa0(1)', 'Child Hobby\xa0_1(1)', 'Child Hobby\xa0_2(1)', 'Child Hobby\xa0_3(1)', 'Child Hobby\xa0_4(1)', 'Child Hobby\xa0_5(1)', 'Child Hobby\xa0_6(1)', 'Child Hobby\xa0_7(1)', 'Child Hobby\xa0_8(1)', 'Child Hobby\xa0_9(1)', 'Child Hobby\xa0_8_TEXT(1)', 'Child Living\xa0_1(1)', 'Child Living\xa0_2(1)', 'Child Living\xa0_3(1)', 'Child Living\xa0_4(1)', 'Child Living\xa0_5(1)', 'Child Living\xa0_6(1)', 'Child Living\xa0_7(1)', 'Child Living\xa0_8(1)', 'Child Living\xa0_9(1)', 'Child Living\xa0_10(1)', 'Child Living\xa0_11(1)', 'Child Living\xa0_12(1)', 'Child Describe\xa0_1(1)', 'Child Describe\xa0_2(1)', 'Child Describe\xa0_3(1)', 'Child Describe\xa0_4(1)', 'Child Describe\xa0_5(1)', 'Child Describe\xa0_6(1)', 'Child Describe\xa0_7(1)', 'Child Describe\xa0_8(1)', 'Child Group\xa0_1(1)', 'Child Group\xa0_2(1)', 'Child Group\xa0_3(1)', 'Child Group\x

In [85]:
stroop_cols = [col for col in df3.columns if col.startswith("Practice") or col.startswith("Base") or col.startswith("Emotional") or (col.startswith("Control") and not col.startswith("Control,"))]
stroop_cols_todrop = stroop_cols

df_S_DG = df_S_STR_DG.drop(columns=stroop_cols_todrop)

stroop_cols.append('ResponseID')
df_stroop = df3[stroop_cols]

print(stroop_cols) 
print(f"Stroop: len({len(df_stroop.columns)})")

print(f"Adult: len({len(df_S_DG.columns)})")

for item in df_S_DG.columns:
    print(item)

['Practice(70)', 'Practice Timing_1(70)', 'Practice Timing_2(70)', 'Practice Timing_3(70)', 'Practice Timing_4(70)', 'Practice(71)', 'Practice Timing_1(71)', 'Practice Timing_2(71)', 'Practice Timing_3(71)', 'Practice Timing_4(71)', 'Practice(72)', 'Practice Timing_1(72)', 'Practice Timing_2(72)', 'Practice Timing_3(72)', 'Practice Timing_4(72)', 'Practice(73)', 'Practice Timing_1(73)', 'Practice Timing_2(73)', 'Practice Timing_3(73)', 'Practice Timing_4(73)', 'Practice(74)', 'Practice Timing_1(74)', 'Practice Timing_2(74)', 'Practice Timing_3(74)', 'Practice Timing_4(74)', 'Practice(75)', 'Practice Timing_1(75)', 'Practice Timing_2(75)', 'Practice Timing_3(75)', 'Practice Timing_4(75)', 'Practice(76)', 'Practice Timing_1(76)', 'Practice Timing_2(76)', 'Practice Timing_3(76)', 'Practice Timing_4(76)', 'Practice(77)', 'Practice Timing_1(77)', 'Practice Timing_2(77)', 'Practice Timing_3(77)', 'Practice Timing_4(77)', 'Practice(78)', 'Practice Timing_1(78)', 'Practice Timing_2(78)', 'Prac

In [86]:
DG_cols = [col for col in df3.columns if ((col.startswith("10-") or col.startswith("RO-BR-FL_") or col.startswith("15-")) and not col in("10-10 10-5", "10-5 10-10", "10-10 10-15", "10-15 10-10", "10-10 15-5", "15-5 10-10", "10-10 15-20", "15-20 10-10"))]
DG_cols_todrop = DG_cols
"""10-10 10-5
10-5 10-10
10-10 10-15
10-15 10-10
10-10 15-5
15-5 10-10
10-10 15-20
15-20 10-10"""

df_S = df_S_DG.drop(columns=DG_cols_todrop)

DG_cols.append('ResponseID')
df_DG = df3[DG_cols]

print(DG_cols) 
print(f"DG: len({len(df_DG.columns)})")
print(f"Adult: len({len(df_S.columns)})")

for item in df_S.columns:
    print(item)

['10-10 10-5 Timing_1', '10-10 10-5 Timing_2', '10-10 10-5 Timing_3', '10-10 10-5 Timing_4', '10-5 10-10 Timing_1', '10-5 10-10 Timing_2', '10-5 10-10 Timing_3', '10-5 10-10 Timing_4', '10-10 10-15 Timing_1', '10-10 10-15 Timing_2', '10-10 10-15 Timing_3', '10-10 10-15 Timing_4', '10-15 10-10 Timing_1', '10-15 10-10 Timing_2', '10-15 10-10 Timing_3', '10-15 10-10 Timing_4', '10-10 15-5 Timing_1', '10-10 15-5 Timing_2', '10-10 15-5 Timing_3', '10-10 15-5 Timing_4', '15-5 10-10 Timing_1', '15-5 10-10 Timing_2', '15-5 10-10 Timing_3', '15-5 10-10 Timing_4', '10-10 15-20 Timing_1', '10-10 15-20 Timing_2', '10-10 15-20 Timing_3', '10-10 15-20 Timing_4', '15-20 10-10 Timing_1', '15-20 10-10 Timing_2', '15-20 10-10 Timing_3', '15-20 10-10 Timing_4', 'RO-BR-FL_1581', 'RO-BR-FL_897', 'RO-BR-FL_892', 'RO-BR-FL_894', 'RO-BR-FL_895', 'RO-BR-FL_896', 'RO-BR-FL_1260', 'RO-BR-FL_1261', 'RO-BR-FL_1266', 'RO-BR-FL_1271', 'RO-BR-FL_1276', 'RO-BR-FL_1334', 'RO-BR-FL_1335', 'RO-BR-FL_1340', 'RO-BR-FL_1345

In [87]:

# Drop some more unnecessary columns

final_drop_cols = [col for col in df_S.columns if col.startswith("Social Practice") or col.startswith("timeout") or col.startswith("selection") or col.startswith("StartLoop") or (col.startswith("Q") and not col.startswith("Q891") and not col.startswith("Q3051_") and not col.startswith("Q_"))]

df_S_cln = df_S.drop(columns=final_drop_cols)

print(final_drop_cols) 

for item in df_S_cln.columns:
    print(item)

['selection', 'StartLoop3', 'timeout3', 'StartLoop4', 'timeout4', 'StartLoop5', 'timeout5', 'StartLoop', 'timeout', 'StartLoop2', 'timeout2', 'StartLoop6', 'timeout6', 'StartLoop7', 'timeout7', 'StartLoop8', 'timeout8', 'StartLoop9', 'timeout9', 'Q416', 'Q412', 'Q2300', 'Q2301', 'Q2303', 'Q3047', 'Q2309', 'Q2326', 'Q2330', 'Q2337', 'Q2369', 'Q2364', 'Q2397', 'Q733', 'Q735', 'Q1105', 'Q736', 'Q1106', 'Q737', 'Q1100', 'Q922', 'Q1097', 'Q2342', 'Q1109', 'Q1110', 'Q1251', 'Q2346', 'Q2813', 'Q1254', 'Q1255', 'Q1396', 'Q2348', 'Q2986', 'Q405', 'Q723', 'Q250', 'Q2293', 'Q879', 'Q884', 'Q885', 'Q407', 'Q738', 'Q258', 'Q2340', 'Q887', 'Q892', 'Q895', 'Q1107', 'Q1399', 'Q1400', 'Q1541', 'Q2350', 'Q1', 'Q2', 'Q3', 'Q4', 'Social Practice 1', 'Social Practice 2', 'Q7', 'Q1544', 'Q1545', 'Q1686', 'Q2352', 'Q1689', 'Q1845', 'Q1831', 'Q2354', 'Q1986', 'Q2145', 'Q2128', 'Q2356', 'Q2398', 'Q2408(1)', 'Q2606(1)', 'Q2688(1)', 'Q2409(1)', 'Q2451(1)', 'Q2456(1)', 'Q2408(2)', 'Q2606(2)', 'Q2688(2)', 'Q2409(2

In [88]:
#only necessary renaming to avoid confusion

df4 = df3.rename(columns={'Q891': 'Compete_Choice', 
                          'Score3': 'StroopPractice1_Score', 
                          'Score4': 'StroopPractice2_Score', 
                          'Score5': 'StroopBaseline_Score',
                          "Score": 'StroopEmotional_Score',
                          "Score2": 'StroopEmotionalCompetitive_Score',
                          "Score6": 'StroopEmotionalChoice_Score',
                          "Score7": 'StroopControl_Score',
                          "Score8": 'StroopControlCompetitive_Score',
                          "Score9": 'StroopControlChoice_Score',
                          "elapsed": 'StroopEmotional_Time',
                          "elapsed2": 'StroopEmotionalCompetitive_Time',
                            "elapsed3": 'StroopPractice1_Time',
                            "elapsed4": 'StroopPractice2_Time',
                            "elapsed5": 'StroopBaseline_Time',
                            "elapsed6": 'StroopEmotionalChoice_Time',
                            "elapsed7": 'StroopControl_Time',
                            "elapsed8": 'StroopControlCompetitive_Time',
                            "elapsed9": 'StroopControlChoice_Time'
                            })


                               
rename_map = {'Q891': 'Compete_Choice', 
                          'Score3': 'StroopPractice1_Score', 
                          'Score4': 'StroopPractice2_Score', 
                          'Score5': 'StroopBaseline_Score',
                          "Score": 'StroopEmotional_Score',
                          "Score2": 'StroopEmotionalCompetitive_Score',
                          "Score6": 'StroopEmotionalChoice_Score',
                          "Score7": 'StroopControl_Score',
                          "Score8": 'StroopControlCompetitive_Score',
                          "Score9": 'StroopControlChoice_Score',
                          "elapsed": 'StroopEmotional_Time',
                          "elapsed2": 'StroopEmotionalCompetitive_Time',
                            "elapsed3": 'StroopPractice1_Time',
                            "elapsed4": 'StroopPractice2_Time',
                            "elapsed5": 'StroopBaseline_Time',
                            "elapsed6": 'StroopEmotionalChoice_Time',
                            "elapsed7": 'StroopControl_Time',
                            "elapsed8": 'StroopControlCompetitive_Time',
                            "elapsed9": 'StroopControlChoice_Time',
                          '10-10 10-5': 'Costless Sharing EL', 
                          '10-5 10-10': 'Costless Sharing ER', 
                          '10-10 10-15': 'Costless Envy EL', 
                          '10-15 10-10': 'Costless Envy ER',
                          '10-10 15-5': 'Sharing EL', 
                          '15-5 10-10': 'Sharing ER',
                          '10-10 15-20': 'Envy EL',
                          '15-20 10-10': 'Envy ER',
                          'TScoreA': 'Anxiety Score',
                          'TScoreD': 'Depression Score',
                          'ASeverity': 'Anxiety Severity',
                          'DSeverity': 'Depression Severity'
                            }


df_S_cln_ren = df_S_cln.rename(columns=rename_map)
#Create the codebook



In [89]:
df_labeling = df.astype(str)
print(df_labeling.columns.tolist())
print(df_labeling["Take care you/child_5_4_TEXT"])

['ResponseID', 'ResponseSet', 'IPAddress', 'StartDate', 'EndDate', 'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalDataReference', 'Finished', 'Status', 'Score-sum', 'Score-weightedAvg', 'Score-weightedStdDev', 'response_id', 'assignment_id', 'besample_id', 'completion_code', 'Q_BallotBoxStuffing', 'Q_RecaptchaScore', 'Q_RelevantIDDuplicate', 'Q_RelevantIDDuplicateScore', 'Q_RelevantIDFraudScore', 'manuallyIDduplicateIP', 'Q_selection', 'selection', 'Survey, Treatment, Prosocial', 'Q_TotalDuration', 'StartLoop3', 'elapsed3', 'timeout3', 'Score3', 'StartLoop4', 'elapsed4', 'timeout4', 'Score4', 'StartLoop5', 'elapsed5', 'timeout5', 'Score5', 'StartLoop', 'elapsed', 'timeout', 'Score', 'StartLoop2', 'elapsed2', 'timeout2', 'Score2', 'StartLoop6', 'elapsed6', 'timeout6', 'Score6', 'Surveys, Control, Prosocial', 'StartLoop7', 'elapsed7', 'timeout7', 'Score7', 'StartLoop8', 'elapsed8', 'timeout8', 'Score8', 'StartLoop9', 'elapsed9', 'timeout9', 'Score9', 'Treatment, Pr

In [90]:

raw_labels = df_labeling.iloc[0].fillna("").to_dict()

labels = {
    rename_map.get(k, k): v
    for k, v in raw_labels.items()
    if k in df.columns or rename_map.get(k) in df.columns
}

df_S_cln_ren.attrs["variable_labels"] = labels


codebook = (
    pd.DataFrame({
        "variable": df_S_cln_ren.columns,
        "label": [labels.get(c, "") for c in df_S_cln_ren.columns],
        "dtype": df_S_cln_ren.dtypes.astype(str)
    })
)



print(raw_labels)

{'ResponseID': 'ResponseID', 'ResponseSet': 'ResponseSet', 'IPAddress': 'IPAddress', 'StartDate': 'StartDate', 'EndDate': 'EndDate', 'RecipientLastName': 'RecipientLastName', 'RecipientFirstName': 'RecipientFirstName', 'RecipientEmail': 'RecipientEmail', 'ExternalDataReference': 'ExternalDataReference', 'Finished': 'Finished', 'Status': 'Status', 'Score-sum': 'Score-sum', 'Score-weightedAvg': 'Score-weightedAvg', 'Score-weightedStdDev': 'Score-weightedStdDev', 'response_id': 'response_id', 'assignment_id': 'assignment_id', 'besample_id': 'besample_id', 'completion_code': 'completion_code', 'Q_BallotBoxStuffing': 'Q_BallotBoxStuffing', 'Q_RecaptchaScore': 'Q_RecaptchaScore', 'Q_RelevantIDDuplicate': 'Q_RelevantIDDuplicate', 'Q_RelevantIDDuplicateScore': 'Q_RelevantIDDuplicateScore', 'Q_RelevantIDFraudScore': 'Q_RelevantIDFraudScore', 'manuallyIDduplicateIP': 'manuallyIDduplicateIP', 'Q_selection': 'Q_selection', 'selection': 'selection', 'Survey, Treatment, Prosocial': 'Survey, Treatmen

In [91]:
print(type(codebook))
print(codebook.head())
for index, row in codebook.iterrows():
    print(row["variable"], row["label"], row["dtype"])  

<class 'pandas.core.frame.DataFrame'>
              variable       label   dtype
ResponseID  ResponseID  ResponseID  object
IPAddress    IPAddress   IPAddress  object
StartDate    StartDate   StartDate  object
EndDate        EndDate     EndDate  object
Finished      Finished    Finished  object
ResponseID ResponseID object
IPAddress IPAddress object
StartDate StartDate object
EndDate EndDate object
Finished Finished object
response_id response_id object
assignment_id assignment_id object
completion_code completion_code object
Q_RecaptchaScore Q_RecaptchaScore object
Q_RelevantIDDuplicate Q_RelevantIDDuplicate object
Q_RelevantIDDuplicateScore Q_RelevantIDDuplicateScore object
Q_RelevantIDFraudScore Q_RelevantIDFraudScore object
manuallyIDduplicateIP manuallyIDduplicateIP object
Survey, Treatment, Prosocial Survey, Treatment, Prosocial object
Q_TotalDuration Q_TotalDuration object
StroopPractice1_Time elapsed3 object
StroopPractice1_Score Score3 object
StroopPractice2_Time elapsed4 obje

In [92]:
print(df_S_cln_ren.columns.tolist())


# cols_2_val = []
for col in df_S_cln_ren.columns:

    print(f"- {col}")
        #print(col)


['ResponseID', 'IPAddress', 'StartDate', 'EndDate', 'Finished', 'response_id', 'assignment_id', 'completion_code', 'Q_RecaptchaScore', 'Q_RelevantIDDuplicate', 'Q_RelevantIDDuplicateScore', 'Q_RelevantIDFraudScore', 'manuallyIDduplicateIP', 'Survey, Treatment, Prosocial', 'Q_TotalDuration', 'StroopPractice1_Time', 'StroopPractice1_Score', 'StroopPractice2_Time', 'StroopPractice2_Score', 'StroopBaseline_Time', 'StroopBaseline_Score', 'StroopEmotional_Time', 'StroopEmotional_Score', 'StroopEmotionalCompetitive_Time', 'StroopEmotionalCompetitive_Score', 'StroopEmotionalChoice_Time', 'StroopEmotionalChoice_Score', 'Surveys, Control, Prosocial', 'StroopControl_Time', 'StroopControl_Score', 'StroopControlCompetitive_Time', 'StroopControlCompetitive_Score', 'StroopControlChoice_Time', 'StroopControlChoice_Score', 'Treatment, Prosocial, Surveys', 'Control, Prosocial, Surveys', 'Prosocial, Treatment, Surveys', 'Prosocial, Control, Surveys', 'completed', 'Anxiety Score', 'Depression Score', 'Anx

In [93]:
print(len(df_child.columns))
print(len(df_stroop.columns))
print(len(df_DG.columns))

642
1605
64


In [94]:
print(df_stroop.columns.tolist())



final_drop_cols = [col for col in df_stroop.columns if col.startswith("Practice(") or col.startswith("Practice 2(") or col.startswith("Base(") or col.startswith("Base Score") or col.startswith("Emotional(") or col.startswith("Emotional Score") or col.startswith("Emotional 2(") or col.startswith("Emotional 2 Score") or col.startswith("Emotional 3(") or col.startswith("Emotional 3 Score") or col.startswith("Control(") or col.startswith("Control Score") or col.startswith("Control 2(") or col.startswith("Control 2 Score") or col.startswith("Control 3(") or col.startswith("Control 3 Score") or col.startswith("Emotional 2 Grade") or col.startswith("Practice Score")]

df_stroop_cln = df_stroop.drop(columns=final_drop_cols)

print(final_drop_cols) 


"""Practice(
Practice 2(
Base(
Base Score
Emotional(
Emotional Score
Emotional 2(
Emotional 2 Score
Emotional 3(
Emotional 3 Score
Control(
Control Score
Control 2(
Control 2 Score
Control 3(
Control 3 Score
"""

['Practice(70)', 'Practice Timing_1(70)', 'Practice Timing_2(70)', 'Practice Timing_3(70)', 'Practice Timing_4(70)', 'Practice(71)', 'Practice Timing_1(71)', 'Practice Timing_2(71)', 'Practice Timing_3(71)', 'Practice Timing_4(71)', 'Practice(72)', 'Practice Timing_1(72)', 'Practice Timing_2(72)', 'Practice Timing_3(72)', 'Practice Timing_4(72)', 'Practice(73)', 'Practice Timing_1(73)', 'Practice Timing_2(73)', 'Practice Timing_3(73)', 'Practice Timing_4(73)', 'Practice(74)', 'Practice Timing_1(74)', 'Practice Timing_2(74)', 'Practice Timing_3(74)', 'Practice Timing_4(74)', 'Practice(75)', 'Practice Timing_1(75)', 'Practice Timing_2(75)', 'Practice Timing_3(75)', 'Practice Timing_4(75)', 'Practice(76)', 'Practice Timing_1(76)', 'Practice Timing_2(76)', 'Practice Timing_3(76)', 'Practice Timing_4(76)', 'Practice(77)', 'Practice Timing_1(77)', 'Practice Timing_2(77)', 'Practice Timing_3(77)', 'Practice Timing_4(77)', 'Practice(78)', 'Practice Timing_1(78)', 'Practice Timing_2(78)', 'Prac

'Practice(\nPractice 2(\nBase(\nBase Score\nEmotional(\nEmotional Score\nEmotional 2(\nEmotional 2 Score\nEmotional 3(\nEmotional 3 Score\nControl(\nControl Score\nControl 2(\nControl 2 Score\nControl 3(\nControl 3 Score\n'

In [95]:
# for item in df_stroop_cln.columns:
#     print(item)



print(df_stroop_cln.columns.tolist())



for col in df_stroop_cln.columns:
    unique_vals = df_stroop_cln[col].unique()
    print(f"{col}")


['Practice Timing_1(70)', 'Practice Timing_2(70)', 'Practice Timing_3(70)', 'Practice Timing_4(70)', 'Practice Timing_1(71)', 'Practice Timing_2(71)', 'Practice Timing_3(71)', 'Practice Timing_4(71)', 'Practice Timing_1(72)', 'Practice Timing_2(72)', 'Practice Timing_3(72)', 'Practice Timing_4(72)', 'Practice Timing_1(73)', 'Practice Timing_2(73)', 'Practice Timing_3(73)', 'Practice Timing_4(73)', 'Practice Timing_1(74)', 'Practice Timing_2(74)', 'Practice Timing_3(74)', 'Practice Timing_4(74)', 'Practice Timing_1(75)', 'Practice Timing_2(75)', 'Practice Timing_3(75)', 'Practice Timing_4(75)', 'Practice Timing_1(76)', 'Practice Timing_2(76)', 'Practice Timing_3(76)', 'Practice Timing_4(76)', 'Practice Timing_1(77)', 'Practice Timing_2(77)', 'Practice Timing_3(77)', 'Practice Timing_4(77)', 'Practice Timing_1(78)', 'Practice Timing_2(78)', 'Practice Timing_3(78)', 'Practice Timing_4(78)', 'Practice Timing_1(79)', 'Practice Timing_2(79)', 'Practice Timing_3(79)', 'Practice Timing_4(79)',

In [96]:
## Rename columns in df_stroop_cln for clarity
df_stroop_cln.columns = df_stroop_cln.columns.str.replace("Timing_1", "First Click", regex=False)
df_stroop_cln.columns = df_stroop_cln.columns.str.replace("Timing_2", "Last Click", regex=False)
df_stroop_cln.columns = df_stroop_cln.columns.str.replace("Timing_3", "Page Submit", regex=False)
df_stroop_cln.columns = df_stroop_cln.columns.str.replace("Timing_4", "Click Count", regex=False)
df_stroop_cln.columns = df_stroop_cln.columns.str.replace("Emotional 2", "Emotional Competitive", regex=False)
df_stroop_cln.columns = df_stroop_cln.columns.str.replace("Emotional 3", "Emotional Choice", regex=False)
df_stroop_cln.columns = df_stroop_cln.columns.str.replace("Control 2", "Control Competitive", regex=False)
df_stroop_cln.columns = df_stroop_cln.columns.str.replace("Control 3", "Control Choice", regex=False)
# df_stroop_cln_ren = df_stroop_cln.rename(columns={""
                                                  

In [97]:

import re

def rename_practice(col, label, backnum):
    match = re.match(rf"^({label} .*?)\((\d+)\)$", col)
    if match:
        base, num = match.groups()
        return f"{base}({int(num) - backnum})"
    return col  # leave all other columns untouched

df_stroop_cln = df_stroop_cln.rename(columns=lambda col: rename_practice(col, "Practice", 69))
df_stroop_cln = df_stroop_cln.rename(columns=lambda col: rename_practice(col, "Base", 69))
df_stroop_cln = df_stroop_cln.rename(columns=lambda col: rename_practice(col, "Emotional Competitive", 36))
df_stroop_cln = df_stroop_cln.rename(columns=lambda col: rename_practice(col, "Emotional Choice", 36))
df_stroop_cln = df_stroop_cln.rename(columns=lambda col: rename_practice(col, "Control", 69))


for col in df_stroop_cln.columns:
    unique_vals = df_stroop_cln[col].unique()
    print(f"{col}")



Practice First Click(1)
Practice Last Click(1)
Practice Page Submit(1)
Practice Click Count(1)
Practice First Click(2)
Practice Last Click(2)
Practice Page Submit(2)
Practice Click Count(2)
Practice First Click(3)
Practice Last Click(3)
Practice Page Submit(3)
Practice Click Count(3)
Practice First Click(4)
Practice Last Click(4)
Practice Page Submit(4)
Practice Click Count(4)
Practice First Click(5)
Practice Last Click(5)
Practice Page Submit(5)
Practice Click Count(5)
Practice First Click(6)
Practice Last Click(6)
Practice Page Submit(6)
Practice Click Count(6)
Practice First Click(7)
Practice Last Click(7)
Practice Page Submit(7)
Practice Click Count(7)
Practice First Click(8)
Practice Last Click(8)
Practice Page Submit(8)
Practice Click Count(8)
Practice First Click(9)
Practice Last Click(9)
Practice Page Submit(9)
Practice Click Count(9)
Practice First Click(10)
Practice Last Click(10)
Practice Page Submit(10)
Practice Click Count(10)
Practice First Click(11)
Practice Last Click(1

In [98]:
df_DG.columns = df_DG.columns.str.replace("Timing_1", "First Click", regex=False)
df_DG.columns = df_DG.columns.str.replace("Timing_2", "Last Click", regex=False)
df_DG.columns = df_DG.columns.str.replace("Timing_3", "Page Submit", regex=False)
df_DG.columns = df_DG.columns.str.replace("Timing_4", "Click Count", regex=False)


final_drop_cols = [col for col in df_DG.columns if col.startswith("RO-BR-FL_")]

df_DG_cln = df_DG.drop(columns=final_drop_cols)


In [99]:
print(df_S_cln_ren.columns.tolist())


for col in df_S_cln_ren.columns:
    print(f"{col}")





['ResponseID', 'IPAddress', 'StartDate', 'EndDate', 'Finished', 'response_id', 'assignment_id', 'completion_code', 'Q_RecaptchaScore', 'Q_RelevantIDDuplicate', 'Q_RelevantIDDuplicateScore', 'Q_RelevantIDFraudScore', 'manuallyIDduplicateIP', 'Survey, Treatment, Prosocial', 'Q_TotalDuration', 'StroopPractice1_Time', 'StroopPractice1_Score', 'StroopPractice2_Time', 'StroopPractice2_Score', 'StroopBaseline_Time', 'StroopBaseline_Score', 'StroopEmotional_Time', 'StroopEmotional_Score', 'StroopEmotionalCompetitive_Time', 'StroopEmotionalCompetitive_Score', 'StroopEmotionalChoice_Time', 'StroopEmotionalChoice_Score', 'Surveys, Control, Prosocial', 'StroopControl_Time', 'StroopControl_Score', 'StroopControlCompetitive_Time', 'StroopControlCompetitive_Score', 'StroopControlChoice_Time', 'StroopControlChoice_Score', 'Treatment, Prosocial, Surveys', 'Control, Prosocial, Surveys', 'Prosocial, Treatment, Surveys', 'Prosocial, Control, Surveys', 'completed', 'Anxiety Score', 'Depression Score', 'Anx

In [100]:
#print(df_S_cln_ren['ASeverity'].unique())


df_S_cln_ren = df_S_cln_ren.drop(columns=['Stroop Practice 2'])



In [101]:
print(df_S_cln_ren.columns.tolist())


for col in df_S_cln_ren.columns:
    print(f"{col}")

['ResponseID', 'IPAddress', 'StartDate', 'EndDate', 'Finished', 'response_id', 'assignment_id', 'completion_code', 'Q_RecaptchaScore', 'Q_RelevantIDDuplicate', 'Q_RelevantIDDuplicateScore', 'Q_RelevantIDFraudScore', 'manuallyIDduplicateIP', 'Survey, Treatment, Prosocial', 'Q_TotalDuration', 'StroopPractice1_Time', 'StroopPractice1_Score', 'StroopPractice2_Time', 'StroopPractice2_Score', 'StroopBaseline_Time', 'StroopBaseline_Score', 'StroopEmotional_Time', 'StroopEmotional_Score', 'StroopEmotionalCompetitive_Time', 'StroopEmotionalCompetitive_Score', 'StroopEmotionalChoice_Time', 'StroopEmotionalChoice_Score', 'Surveys, Control, Prosocial', 'StroopControl_Time', 'StroopControl_Score', 'StroopControlCompetitive_Time', 'StroopControlCompetitive_Score', 'StroopControlChoice_Time', 'StroopControlChoice_Score', 'Treatment, Prosocial, Surveys', 'Control, Prosocial, Surveys', 'Prosocial, Treatment, Surveys', 'Prosocial, Control, Surveys', 'completed', 'Anxiety Score', 'Depression Score', 'Anx

In [102]:
print(codebook.columns.tolist())
codebook_cln = codebook[codebook["variable"].isin(df_S_cln_ren.columns)].reset_index(drop=True)


['variable', 'label', 'dtype']


In [103]:


for index, row in codebook_cln.iterrows():
    print(f"Variable: {row['variable']}, Label: {row['label']}")

Variable: ResponseID, Label: ResponseID
Variable: IPAddress, Label: IPAddress
Variable: StartDate, Label: StartDate
Variable: EndDate, Label: EndDate
Variable: Finished, Label: Finished
Variable: response_id, Label: response_id
Variable: assignment_id, Label: assignment_id
Variable: completion_code, Label: completion_code
Variable: Q_RecaptchaScore, Label: Q_RecaptchaScore
Variable: Q_RelevantIDDuplicate, Label: Q_RelevantIDDuplicate
Variable: Q_RelevantIDDuplicateScore, Label: Q_RelevantIDDuplicateScore
Variable: Q_RelevantIDFraudScore, Label: Q_RelevantIDFraudScore
Variable: manuallyIDduplicateIP, Label: manuallyIDduplicateIP
Variable: Survey, Treatment, Prosocial, Label: Survey, Treatment, Prosocial
Variable: Q_TotalDuration, Label: Q_TotalDuration
Variable: StroopPractice1_Time, Label: elapsed3
Variable: StroopPractice1_Score, Label: Score3
Variable: StroopPractice2_Time, Label: elapsed4
Variable: StroopPractice2_Score, Label: Score4
Variable: StroopBaseline_Time, Label: elapsed5
V

In [104]:
df_stroop_cln.to_csv("Stroop Test Data Ukraine USFCA 2_2026.csv", index=False, encoding="utf-8-sig")
df_S_cln_ren.to_csv("Survey Data Ukraine USFCA 2_2026.csv", index=False, encoding="utf-8-sig")
df_DG_cln.to_csv("Dictator Game Data Ukraine USFCA 2_2026.csv", index=False, encoding="utf-8-sig")
codebook_cln.to_csv("Codebook for Survey Data Ukraine USFCA 2_2026.csv", index=False, encoding="utf-8-sig")


In [32]:
#df_stroop_cln
#df_S_cln_ren
#df_DG_cln

# STATA CLEANING CODE
# ///clean and sort
# drop in 1
# drop if Finished!="True" | Age =="" | Q891=="" 
# drop if assignment_id == ""
# drop if assignment_id == "undefined"
# drop if completion_code == ""
# drop if completion_code == "undefined"
# drop if ResponseId == ""
# drop if ResponseId == "undefined"
# drop if response_id == "" 
# drop if response_id == "undefined"

# gen numericDate = clock( RecordedDate, "MDYhm")
# sort numericDate
# duplicates drop completion_code, force
# duplicates drop assignment_id , force
# duplicates drop ResponseId, force
# duplicates drop response_id, force
# ///selection///
# duplicates drop IPAddress Q3051_Browser Q3051_Version Q3051_OperatingSystem Q3051_Resolution LocationLatitude LocationLongitude, force

# ///



